In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from definitions import *

In [ ]:
R = 10000e-10 # m
z = np.linspace(0,7e-6,1000)
xi = z / R
G_xi = G(z, R)
G_xi_2 = G(z, 2 * R)

plt.plot(z*1e6,G_xi)
plt.plot(z*1e6,G_xi_2)
plt.xlabel('z [um]')
plt.ylabel('G(z)')
plt.grid()

In [ ]:
R = 10000e-10 # m
Rs = [50e-9, 100e-9, 200e-9]
for R in Rs:
    z = np.linspace(0,500e-9,1000)
    G_xi = G_norm(z, R)
    # G_xi_2 = G_norm(z, 2 * R)

    plt.plot(z*1e9,G_xi, label=rf'$R = {round(R * 1e9)}$ nm')
    # plt.plot(z*1e6,G_xi_2)
plt.xlabel(r'$\delta$ [nm]')
plt.ylabel(r'$G_0(\delta)$')
plt.xlim((0,500))
plt.grid()
plt.legend()
plt.savefig("docs/analytical-G0.eps",bbox_inches="tight", pad_inches=0, format='eps')

In [ ]:
P = P_analytical(z, R, t=1e-3, wavelength=2.165e-10)
plt.plot(z*1e6,P)
plt.xlabel('z [um]')
plt.ylabel('P(z)')
plt.grid()

In [ ]:
z_sim, P_sim = np.genfromtxt(f'simulation-fit.txt', delimiter=',',usecols=(0,1),unpack=True)

plt.plot(z*1e6,P, label='Analytical')
plt.plot(z_sim*1e6,P_sim,'.', label='Fitted to simulation')
plt.xlabel('z [um]')
plt.ylabel('P(z)')
plt.legend()
plt.grid()

# Analytical form factor
Dilute SANS samples have $Q$-ranges corresponding to their form factor

In [ ]:
def form_factor(Q, R):
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2

Rs = [50e-9, 300e-9, 2000e-9]
Q = np.linspace(0, 0.005e10,1000000)
# for R in Rs:
R = 300e-9
ff = form_factor(Q,R)
plt.plot(Q * 1e-10,ff)
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel(r'$Q$ [Å$^{-1}$]')
plt.ylabel(r'$P(Q)$')
plt.xlim((0.0,0.005))
plt.ylim((1e-8,1.1))
plt.axvline(0.3 / R * 1e-10, linestyle='--', color='red', label=r'$Q = \frac{0.3}{R}$')
plt.axvline(10 / R * 1e-10, linestyle='--', color='red', label=r'$Q = \frac{10}{R}$')
plt.legend()
plt.grid()